In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pip install Pillow 


In [ ]:
!pip install facenet_pytorch==2.5.2


In [ ]:
conda update seaborn statsmodels joblib


In [ ]:
import os
import zipfile
import numpy as np
import pandas as pd
import matplotlib
import seaborn as sns
import torch
import matplotlib.pyplot as plt
# from tqdm import tqdm_notebook
%matplotlib inline 
# from google.colab.patches import cv2_imshow
from IPython.display import HTML #imports to play videos
from base64 import b64encode 
import cv2 as cv
from skimage.measure import compare_ssim
import glob
import time
from PIL import Image
from facenet_pytorch import MTCNN, InceptionResnetV1, extract_face
from tqdm import tqdm

import math
import pickle
from functools import partial
from collections import defaultdict

from PIL import Image
from glob import glob

import cv2
import skimage.measure
import albumentations as A
from tqdm.notebook import tqdm 
from albumentations.pytorch import ToTensor 

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision.models.video import mc3_18, r2plus1d_18

from facenet_pytorch import MTCNN

In [ ]:
!pip install -q imbalanced-learn


In [ ]:
import json
import os
import numpy as np
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split

# Load the metadata from the uploaded file
metadata_path = '/kaggle/input/metadata2/metadata.json'  # Update this to your specific metadata path

with open(metadata_path) as f:
    metadata = json.load(f)

# Define the path to the videos in Kaggle's input directory
videos_dir = '/kaggle/input/deepfake-detection-videos/All_Videos'  # Update this to your specific dataset folder name

# Separate FAKE and REAL videos
X = []  # Video filenames
y = []  # Labels

# Store unique original videos to avoid duplicates
originals_seen = set()

for video, details in metadata.items():
    video_path = os.path.join(videos_dir, video)
    if os.path.exists(video_path):  # Ensure the video file exists in the dataset
        if details['label'] == 'FAKE' and details['original'] not in originals_seen:
            X.append(video_path)
            y.append(0)  # Label 0 for FAKE
            originals_seen.add(details['original'])
        elif details['label'] == 'REAL' and video not in originals_seen:
            X.append(video_path)
            y.append(1)  # Label 1 for REAL
            originals_seen.add(video)

# Convert to numpy arrays for compatibility with imbalanced-learn
X = np.array(X).reshape(-1, 1)
y = np.array(y)

# Apply Random Oversampling
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X, y)

# Convert back to list for further processing
X_resampled = X_resampled.flatten().tolist()

# Optionally split into train/validation
train_videos, val_videos, y_train, y_val = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Save the final lists if needed
os.makedirs('/kaggle/working', exist_ok=True)
with open('/kaggle/working/train_videos.json', 'w') as f:
    json.dump(train_videos, f)
    
with open('/kaggle/working/val_videos.json', 'w') as f:
    json.dump(val_videos, f)

print(f"Train set size: {len(train_videos)}, Validation set size: {len(val_videos)}")


In [ ]:
DATA_FOLDER = "../input/deepfake-detection-challenge" 
DATA_FOLDER2 = "/kaggle/input/deep-fake-detection-dfd-entire-original-dataset" 
TRAIN_SAMPLE_FOLDER = "/kaggle/input/deep-fake-detection-dfd-entire-original-dataset"
TEST_FOLDER = "test_videos"

In [ ]:
train_list = list(os.listdir(os.path.join(DATA_FOLDER2, TRAIN_SAMPLE_FOLDER)))
ext_dict = []
for file in train_list:
    file_ext = file.split('.')[1]
    if (file_ext not in ext_dict):
        ext_dict.append(file_ext)
print(f"Extensions: {ext_dict}") 

In [ ]:
FACE_DETECTION_FOLDER = '../input/haarcascades'
print(f"Face detection resources: {os.listdir(FACE_DETECTION_FOLDER)}")    

In [ ]:
test_list = list(os.listdir(os.path.join(DATA_FOLDER, TEST_FOLDER)))
ext_dict = []
for file in test_list:
    file_ext = file.split('.')[1]
    if (file_ext not in ext_dict):
        ext_dict.append(file_ext)
print(f"Extensions: {ext_dict}")


In [ ]:
json_file = '/kaggle/input/metadata2/metadata.json'
print(f"JSON file: {json_file}")
#reading the json file
def get_meta_from_json(path):
    df = pd.read_json(os.path.join(DATA_FOLDER, path, json_file))
    df = df.T
    return df

meta_train_df = get_meta_from_json('/kaggle/input/metadata2/metadata.json')
meta_train_df.head()


In [ ]:
def missing_data(data):
    total = data.isnull().sum()
    percent = (data.isnull().sum()/data.isnull().count()*100)
    tt = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    types = []
    for col in data.columns:
        dtype = str(data[col].dtype)
        types.append(dtype)
    tt['Types'] = types
    return(np.transpose(tt))

In [ ]:
missing_data(meta_train_df)

In [ ]:
missing_data(meta_train_df.loc[meta_train_df.label == 'REAL'])

In [ ]:
def unique_values(data):
    total = data.count()
    tt = pd.DataFrame(total)
    tt.columns = ['Totals']
    uniques = []
    for col in data.columns:
        unique = data[col].nunique() #collect all unique instances
        uniques.append(unique)
    tt['Uniques'] = uniques
    return(np.transpose(tt))

In [ ]:
unique_values(meta_train_df)

In [ ]:
def most_frequent_values(data):
    total = data.count()
    tt = pd.DataFrame(total)
    tt.columns = ['Total']
    items = []
    vals = []
    for col in data.columns:
        itm = data[col].value_counts().index[0]
        val = data[col].value_counts().values[0]
        items.append(itm)
        vals.append(val)
    tt['Most frequent item'] = items
    tt['Frequence'] = vals
    tt['Percent from total'] = np.round(vals / total * 100, 3)
    return(np.transpose(tt))


In [ ]:
most_frequent_values(meta_train_df)

In [ ]:
most_frequent_values(meta_train_df.loc[meta_train_df.label == 'FAKE'])

In [ ]:
def plot_count(feature, title, df, size=1):
  '''
    Plot count of classes / feature
    param: feature - the feature to analyze
    param: title - title to add to the graph
    param: df - dataframe from which we plot feature's classes distribution 
    param: size - default 1.
  '''  
  f, ax = plt.subplots(1,1, figsize=(4*size,4))
  total = float(len(df))
  g =  sns.countplot(df[feature], order = df[feature].value_counts().index[:20], palette='Set3')
  g.set_title("Number and percentage of {}".format(title)) 
  if(size > 2):
    plt.xticks(rotation=90, size=8)
  for p in ax.patches:
     height = p.get_height()
     ax.text(p.get_x()+ p.get_width()/2.,height + 3,'{:1.2f}%'.format(100*height/total),ha="center")

  plt.show()


In [ ]:
plot_count('split','split(train)',meta_train_df)


In [ ]:
plot_count('label','label(train)',meta_train_df)

In [ ]:
meta = np.array(list(meta_train_df.index))
storage = np.array([file for file in train_list if  file.endswith('mp4')])
print(f"Metadata: {meta.shape[0]}, Folder: {storage.shape[0]}")
print(f"Files in metadata and not in folder: {np.setdiff1d(meta,storage,assume_unique=False).shape[0]}")
print(f"Files in folder and not in metadata: {np.setdiff1d(storage,meta,assume_unique=False).shape[0]}")

In [ ]:
fake_train_sample_video = list(meta_train_df.loc[meta_train_df.label=='FAKE'].sample(3).index)
fake_train_sample_video

In [ ]:
def display_image_from_video(video_path):
    '''
    input: video_path - path for video
    process:
    1. perform a video capture from the video
    2. read the image
    3. display the image
    '''
    capture_img = cv.VideoCapture(video_path)
    ret, frame = capture_img.read()
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111)
    frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    ax.imshow(frame)

In [ ]:
for video_file in fake_train_sample_video:
  display_image_from_video(os.path.join(DATA_FOLDER,TRAIN_SAMPLE_FOLDER,video_file))

In [ ]:
real_train_sample_video = list(meta_train_df.loc[meta_train_df.label=='REAL'].sample(3).index) #viewing the real videos
real_train_sample_video

In [ ]:
for video in real_train_sample_video:
  display_image_from_video(os.path.join(DATA_FOLDER,TRAIN_SAMPLE_FOLDER,video))

In [ ]:
def display_image_from_video_list(video_path_list, video_folder=TRAIN_SAMPLE_FOLDER):
    '''
    input: video_path_list - path for video
    process:
    0. for each video in the video path list
        1. perform a video capture from the video
        2. read the image
        3. display the image
    '''
    plt.figure()
    fig, ax = plt.subplots(2,3,figsize=(16,8))
    #we only show images extracted from first 6 videos
    for i, video_file in enumerate(video_path_list[0:6]):
      video_path = os.path.join(DATA_FOLDER, video_folder, video_file)
      capture_img = cv.VideoCapture(video_path)
      ret, frame = capture_img.read()
      frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
      ax[i//3, i%3].imshow(frame)
      ax[i//3, i%3].set_title(f"Video: {video_file}")
      ax[i//3, i%3].axis('on')

In [ ]:
same_original_fake_train_sample_video = list(meta_train_df.loc[meta_train_df.original=='aabdnomlru.mp4'].index)
display_image_from_video_list(same_original_fake_train_sample_video)

In [ ]:
!tar xvf /kaggle/input/ffmpeg-static-build/ffmpeg-git-amd64-static.tar.xz

In [ ]:
import pandas as pd
import numpy as np
import glob, shutil
import timeit, os, gc
import subprocess as sp
from tqdm import tqdm
from collections import defaultdict
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
import json
from IPython.display import HTML
from base64 import b64encode
import cv2
%matplotlib inline

In [ ]:
HOME = "./"
FFMPEG = "/kaggle/working/ffmpeg-git-20191209-amd64-static"
FFMPEG_PATH = FFMPEG
DATA_FOLDER = "/kaggle/input/deepfake-detection-videos"
TMP_FOLDER = HOME
DATA_FOLDER_TRAIN = DATA_FOLDER
VIDEOS_FOLDER_TRAIN = DATA_FOLDER_TRAIN + "/All_Videos"
IMAGES_FOLDER_TRAIN = TMP_FOLDER + "/images"
AUDIOS_FOLDER_TRAIN = TMP_FOLDER + "/audios"
EXTRACT_META = True # False
EXTRACT_CONTENT = True # False
EXTRACT_FACES = True # False
FRAME_RATE = 0.5 # Frame per
print(FFMPEG)

In [ ]:
!pip install imageio-ffmpeg==0.4.3


In [ ]:
import imageio_ffmpeg as ffmpeg
print(ffmpeg.get_ffmpeg_version())


In [ ]:
def run_command(*popenargs, **kwargs):
    closeNULL = 0
    try:
        from subprocess import DEVNULL
        closeNULL = 0
    except ImportError:
        import os
        DEVNULL = open(os.devnull, 'wb')
        closeNULL = 1

    process = sp.Popen(stdout=sp.PIPE, stderr=DEVNULL, *popenargs, **kwargs)
    output, unused_err = process.communicate()
    retcode = process.poll()

    if closeNULL:
        DEVNULL.close()

    if retcode:
        cmd = kwargs.get("args")
        if cmd is None:
            cmd = popenargs[0]
        error = sp.CalledProcessError(retcode, cmd)
        error.output = output
        raise error
    return output

def ffprobe(filename, options = ["-show_error", "-show_format", "-show_streams", "-show_programs", "-show_chapters", "-show_private_data"]):
    ret = {}
    command = [FFMPEG_PATH + "/ffprobe", "-v", "error", *options, "-print_format", "json", filename]
    ret = run_command(command)
    if ret:
        ret = json.loads(ret)
    return ret

# ffmpeg -i input.mov -r 0.25 output_%04d.png
def ffextract_frames(filename, output_folder, rate = 0.25):
    command = [FFMPEG_PATH + "/ffmpeg", "-i", filename, "-r", str(rate), "-y", output_folder + "/output_%04d.png"]
    ret = run_command(command)
    return ret

# ffmpeg -i input-video.mp4 output-audio.mp3
def ffextract_audio(filename, output_path):
    command = [FFMPEG_PATH + "/ffmpeg", "-i", filename, "-vn", "-ac", "1", "-acodec", "copy", "-y", output_path]
    ret = run_command(command)
    return ret

In [ ]:
import subprocess as sp
import json
import os

FFMPEG_PATH = "/kaggle/working/ffmpeg-git-20191209-amd64-static"

def run_command(*popenargs, **kwargs):
    closeNULL = 0
    try:
        from subprocess import DEVNULL
        closeNULL = 0
    except ImportError:
        import os
        DEVNULL = open(os.devnull, 'wb')
        closeNULL = 1

    process = sp.Popen(stdout=sp.PIPE, stderr=DEVNULL, *popenargs, **kwargs)
    output, unused_err = process.communicate()
    retcode = process.poll()

    if closeNULL:
        DEVNULL.close()

    if retcode:
        cmd = kwargs.get("args", popenargs[0] if popenargs else None)
        error = sp.CalledProcessError(retcode, cmd)
        error.output = output
        raise error
    return output

def ffprobe(filename, options=None):
    if options is None:
        options = ["-show_error", "-show_format", "-show_streams", "-show_programs", "-show_chapters", "-show_private_data"]
    command = [FFMPEG_PATH + "/ffprobe", "-v", "error", *options, "-print_format", "json", filename]
    ret = run_command(command)
    if ret:
        ret = json.loads(ret)
    return ret

def ffextract_frames(filename, output_folder, rate=0.25):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    command = [FFMPEG_PATH + "/ffmpeg", "-i", filename, "-r", str(rate), "-y", os.path.join(output_folder, "output_%04d.png")]
    ret = run_command(command)
    return ret

def ffextract_audio(filename, output_path):
    command = [FFMPEG_PATH + "/ffmpeg", "-i", filename, "-vn", "-ac", "1", "-acodec", "copy", "-y", output_path]
    ret = run_command(command)
    return ret


In [ ]:
import os

subfolder = DATA_FOLDER_TRAIN + "/All_Videos"  # Replace with the correct path
print(f"Looking for files in: {subfolder}")

if os.path.isdir(subfolder):
    all_files = os.listdir(subfolder)
    print(f"All files in directory: {all_files}")
else:
    print(f"Directory does not exist: {subfolder}")


In [ ]:
if EXTRACT_META == True:
    results = []
    subfolder = VIDEOS_FOLDER_TRAIN
    filepaths = glob.glob(subfolder + "/*.mp4")
    for filepath in tqdm(filepaths):
        js = ffprobe(filepath)
#        print(js)
        if js:
            results.append(
                (js.get("format", {}).get("filename")[len(subfolder) + 1:],
                js.get("format", {}).get("format_long_name"),
                # Video 
                js.get("streams", [{}, {}])[0].get("codec_name"),
                js.get("streams", [{}, {}])[0].get("height"),
                js.get("streams", [{}, {}])[0].get("width"),
                js.get("streams", [{}, {}])[0].get("nb_frames"),
                js.get("streams", [{}, {}])[0].get("bit_rate"),
                js.get("streams", [{}, {}])[0].get("duration"),
                js.get("streams", [{}, {}])[0].get("start_time"),
                js.get("streams", [{}, {}])[0].get("avg_frame_rate"),
                 # Audio
                js.get("streams", [{}, {}])[1].get("codec_name"),
                js.get("streams", [{}, {}])[1].get("channels"),
                js.get("streams", [{}, {}])[1].get("sample_rate"),
                js.get("streams", [{}, {}])[1].get("nb_frames"),
                js.get("streams", [{}, {}])[1].get("bit_rate"),
                js.get("streams", [{}, {}])[1].get("duration"),
                js.get("streams", [{}, {}])[1].get("start_time")),
            )

    meta_pd = pd.DataFrame(results, columns=["filename", "format", "video_codec_name", "video_height", "video_width",
                                            "video_nb_frames", "video_bit_rate", "video_duration", "video_start_time","video_fps",
                                            "audio_codec_name", "audio_channels", "audio_sample_rate", "audio_nb_frames",
                                            "audio_bit_rate", "audio_duration", "audio_start_time"])
    meta_pd["video_fps"] = meta_pd["video_fps"].apply(lambda x: float(x.split("/")[0])/float(x.split("/")[1]) if len(x.split("/")) == 2 else None)
    meta_pd["video_duration"] = meta_pd["video_duration"].astype(np.float32)
    meta_pd["video_bit_rate"] = meta_pd["video_bit_rate"].astype(np.float32)
    meta_pd["video_start_time"] = meta_pd["video_start_time"].astype(np.float32)
    meta_pd["video_nb_frames"] = meta_pd["video_nb_frames"].astype(np.float32)
    meta_pd["video_bit_rate"] = meta_pd["video_bit_rate"].astype(np.float32)
    meta_pd["audio_sample_rate"] = meta_pd["audio_sample_rate"].astype(np.float32)
    meta_pd["audio_nb_frames"] = meta_pd["audio_nb_frames"].astype(np.float32)
    meta_pd["audio_bit_rate"] = meta_pd["audio_bit_rate"].astype(np.float32)
    meta_pd["audio_duration"] = meta_pd["audio_duration"].astype(np.float32)
    meta_pd["audio_start_time"] = meta_pd["audio_start_time"].astype(np.float32)
    meta_pd.to_pickle(HOME + "videos_meta.pkl")
else:
    meta_pd = pd.read_pickle(HOME + "videos_meta.pkl")
meta_pd.head()

In [ ]:
fig, ax = plt.subplots(1,6, figsize=(22, 3))
d = sns.distplot(meta_pd["video_fps"], ax=ax[0])
d = sns.distplot(meta_pd["video_duration"], ax=ax[1])
d = sns.distplot(meta_pd["video_width"], ax=ax[2])
d = sns.distplot(meta_pd["video_height"], ax=ax[3])
d = sns.distplot(meta_pd["video_nb_frames"], ax=ax[4])
d = sns.distplot(meta_pd["video_bit_rate"], ax=ax[5])

In [ ]:
train_pd = pd.read_json(VIDEOS_FOLDER_TRAIN + "/metadata.json").T.reset_index().rename(columns={"index": "filename"})
train_pd.head()

In [ ]:
train_pd = pd.read_json(VIDEOS_FOLDER_TRAIN + "/metadata.json").T.reset_index().rename(columns={"index": "filename"})
train_pd.head()

In [ ]:
train_pd = pd.merge(train_pd, meta_pd[["filename", "video_height", "video_width", "video_nb_frames", "video_bit_rate", "audio_nb_frames"]], on="filename", how="left")
train_pd["count"] = train_pd.groupby(["original"])["original"].transform('count')
# train_pd.to_pickle(HOME + "train_meta.pkl")
train_pd.head()

In [ ]:
AUDIO_FORMAT = "aac" # "wav"
videos_folder = VIDEOS_FOLDER_TRAIN
images_folder_path = IMAGES_FOLDER_TRAIN
audios_folder_path = AUDIOS_FOLDER_TRAIN
if EXTRACT_CONTENT == True:
    # 1h20min for chunk#0 (11GB)
    # Extract some images + audio track
    for idx, row in tqdm(train_pd.iterrows(), total=meta_pd.shape[0]):
        try:
            video_path = videos_folder + "/" + row["filename"]
            images_path = images_folder_path + "/" + row["filename"][:-4]
            audio_path = audios_folder_path + "/" + row["filename"][:-4]
            # Extract images
            if not os.path.exists(images_path): os.makedirs(images_path)
            ret = ffextract_frames(video_path, images_path, rate = FRAME_RATE)
            # Extract audio
            if not os.path.exists(audio_path): os.makedirs(audio_path)
            # ret = ffextract_audio(video_path, audio_path + "/audio." + AUDIO_FORMAT)
        except:
            print("Cannot extract frames/audio for:" + row["filename"])

In [ ]:
train_pd.tail()

In [ ]:
idx = 21 # 27 # 21 # 19 # 12 # 6
fake = train_pd["filename"][idx]
real = train_pd["original"][idx]
vid_width = train_pd["video_width"][idx]
vid_real = open(VIDEOS_FOLDER_TRAIN + "/" + real, 'rb').read()
data_url_real = "data:video/mp4;base64," + b64encode(vid_real).decode()
vid_fake = open(VIDEOS_FOLDER_TRAIN + "/" + fake, 'rb').read()
data_url_fake = "data:video/mp4;base64," + b64encode(vid_fake).decode()
HTML("""
<div style='width: 100%%; display: table;'>
    <div style='display: table-row'>
        <div style='width: %dpx; display: table-cell;'><b>Real</b>: %s<br/><video width=%d controls><source src="%s" type="video/mp4"></video></div>
        <div style='display: table-cell;'><b>Fake</b>: %s<br/><video width=%d controls><source src="%s" type="video/mp4"></video></div>
    </div>
</div>
""" % ( int(vid_width/3.2) + 10, 
       real, int(vid_width/3.2), data_url_real, 
       fake, int(vid_width/3.2), data_url_fake))

In [ ]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

def detect_face_cv2(img):
    # Move to grayscale
    gray_img = cv2.cvtColor(img.copy(), cv2.COLOR_RGB2GRAY)
    face_locations = []
    face_rects = face_cascade.detectMultiScale(gray_img, scaleFactor=1.3, minNeighbors=5)     
    for (x,y,w,h) in face_rects: 
        face_location = (x,y,w,h)
        face_locations.append((face_location, 1.0))
    return face_locations

In [ ]:
pip install mtcnn

In [ ]:
from mtcnn import MTCNN
detector = MTCNN()

def detect_face_mtcnn(img):
    face_locations = []
    items = detector.detect_faces(img)
    for face in items:
        face_location = tuple(face.get('box'))
        face_confidence = float(face.get('confidence'))
        face_locations.append((face_location, face_confidence))
    return face_locations

In [ ]:
def extract_faces(files, source, detector=detect_face_cv2):
    results = []
    # for idx, file in tqdm(enumerate(files), total=len(files)):
    for idx, file in enumerate(files):
        try:
            img = cv2.cvtColor(cv2.imread(file, cv2.IMREAD_UNCHANGED), cv2.COLOR_BGR2RGB)
            face_locations = detector(img)
            results.append((source, file[file.find("output_"):], face_locations, len(face_locations)))
        except:
            print("Cannot extract faces for image: %s" % file)
    return results

In [ ]:
file = fake
dump_folder = images_folder_path + "/" + file[:-4]
files = glob.glob(dump_folder + "/*")
DETECTORS = {
    "cv2": detect_face_cv2,
    "mtcnn": detect_face_mtcnn
}
faces_pd = None
for key, value in DETECTORS.items():
    tmp_pd = pd.DataFrame(extract_faces(files, file, detector=value), columns=["filename", "image", "boxes_" + key , "faces_" + key])
    if faces_pd is None:
        faces_pd = tmp_pd
    else:
        faces_pd = pd.merge(faces_pd, tmp_pd, on=["filename", "image"], how="left")
faces_pd.head(12)


In [ ]:
def plot_faces_boxes(df, max_cols = 2, max_rows = 6, fsize=(24, 5), max_items=12):    
    idx = 0    
    for item_idx, item in df.iterrows():
        img = cv2.cvtColor(cv2.imread(IMAGES_FOLDER_TRAIN + "/" + item["filename"][:-4] +"/" + item["image"], cv2.IMREAD_UNCHANGED), cv2.COLOR_BGR2RGB)    
        face_img = img #.copy()
        # grid subplots
        row = idx // max_cols
        col = idx % max_cols
        if col == 0: fig = plt.figure(figsize=fsize)
        ax = fig.add_subplot(1, max_cols, col + 1)
        ax.axis("off")
        # display image with boxes
        cols = [c for c in df.columns if "boxes" in c]
        for i, c in enumerate(cols, 0):
            face_locations = item[c]
            face_confidence = item[c]            
            if len(face_locations) > 0:
                for face_location in face_locations:        
                    ((x,y,w,h), confidence) = face_location
                    # face_img = face_img[y:y+h, x:x+w]
                    cv2.rectangle(face_img, (x, y), (x+w, y+h), (255,i*255,0), 8)
                    cv2.putText(face_img, '%.1f' % (confidence*100.0), (x+w, y+h), cv2.FONT_HERSHEY_SIMPLEX, 2.0, (255,i*255,0), 9, cv2.LINE_AA)
                ax.imshow(face_img)
            else:
                ax.imshow(img)
            ax.set_title("%s %s / %s - Faces: %d %s %s" % (item["label"] if "label" in df.columns else "", 
                                                           item["filename"], item["image"],
                                                           item["faces_mtcnn"] if "faces_mtcnn" in df.columns else len(face_locations),
                                                           item["faces_mtcnn_median"] if "faces_mtcnn_median" in df.columns else "",
                                                           item["faces"] if "faces" in df.columns else ""))
        if (col == max_cols -1): plt.show()
        idx = idx + 1
        if (max_items > 0 and idx >=max_items): break

In [ ]:
plot_faces_boxes(faces_pd)

In [ ]:
def run_detector_on_video(videos_filename, verbose=False):
    if verbose == True: 
        print("Starting with batch of %d videos" % len(videos_filename))
    tmp_faces_pd = None
    for file in videos_filename:
        # Find out dump folder with images
        dump_folder = images_folder_path + "/" + file[:-4]
        # List files
        files = glob.glob(dump_folder + "/*")
        DETECTORS = {
            "mtcnn": detect_face_mtcnn
        }
        for key, value in DETECTORS.items():
            tmp_pd = pd.DataFrame(extract_faces(files, file, detector=value), columns=["filename", "image", "boxes_" + key , "faces_" + key])
            if tmp_faces_pd is None:
                tmp_faces_pd = tmp_pd
            else:
                tmp_faces_pd = pd.concat([tmp_faces_pd, tmp_pd], axis=0)
    return tmp_faces_pd

In [ ]:
import multiprocessing
cpus = multiprocessing.cpu_count()
if EXTRACT_FACES == True:
    resultfutures = []
    results = []
    tasks = np.array_split(train_pd["filename"].unique(), 20)
    print("Tasks: %d" % len(tasks))
    with ThreadPoolExecutor(max_workers=cpus) as executor:
        resultfutures = tqdm(executor.map(run_detector_on_video, tasks), total=len(tasks))
    results = [x for x in resultfutures]
    executor.shutdown()
    # Gather results
    all_faces_pd = None
    for result in results:
        if all_faces_pd is None:
            all_faces_pd = result
        else:
            all_faces_pd = pd.concat([all_faces_pd, result], axis=0)
    all_faces_pd = all_faces_pd.reset_index(drop=True)
    all_faces_pd.to_pickle(HOME + "faces.pkl")
else:
    all_faces_pd = pd.read_pickle(HOME + "faces.pkl")
print(all_faces_pd.shape)

In [ ]:
all_faces_pd["faces_mtcnn_avg"] = all_faces_pd.groupby("filename")["faces_mtcnn"].transform(np.nanmean)
all_faces_pd["faces_mtcnn_median"] = all_faces_pd.groupby("filename")["faces_mtcnn"].transform(np.nanmedian)
all_faces_pd.head()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(22, 3))
d = sns.distplot(all_faces_pd["faces_mtcnn_avg"], kde=True, ax=ax[0])
d = sns.distplot(all_faces_pd["faces_mtcnn_median"], kde=False, ax=ax[1])

In [ ]:
plot_faces_boxes(all_faces_pd[all_faces_pd["faces_mtcnn"] == 3], max_items=24)

In [ ]:
clean_faces_pd = pd.merge(all_faces_pd, train_pd, on="filename", how="left")
clean_faces_pd.head()

In [ ]:
def faces_max_item(boxes, idx1, idx2):
    ret = 0
    if len(boxes) > 0:
        ret = max(boxes, key=lambda item: item[idx1][idx2])[idx1][idx2]
    return ret

def faces_max_confidence(boxes):
    ret = 0
    if len(boxes) > 0:
        ret = max(boxes, key=lambda item: item[1])[1]
    return ret

def faces_min_confidence(boxes):
    ret = 0
    if len(boxes) > 0:
        ret = min(boxes, key=lambda item: item[1])[1]
    return ret

clean_faces_pd["faces_max_width"] = clean_faces_pd["boxes_mtcnn"].apply(lambda x: faces_max_item(x, 0, 2)) 
clean_faces_pd["faces_max_height"] = clean_faces_pd["boxes_mtcnn"].apply(lambda x: faces_max_item(x, 0, 3))
clean_faces_pd["faces_max_conf"] = clean_faces_pd["boxes_mtcnn"].apply(lambda x: faces_max_confidence(x))
clean_faces_pd["faces_min_conf"] = clean_faces_pd["boxes_mtcnn"].apply(lambda x: faces_min_confidence(x))

In [ ]:
print("Faces stats:")
print(clean_faces_pd[["faces_max_width", "faces_max_height", "faces_min_conf", "faces_max_conf"]].describe(percentiles=[0.01,0.05, 0.1,0.25,0.5,0.75,0.9,0.95,0.99]))
fig, ax = plt.subplots(1, 2, figsize=(22, 3))
d = sns.distplot(clean_faces_pd["faces_max_width"], kde=True, ax=ax[0])
d = sns.distplot(clean_faces_pd["faces_max_height"], kde=True, ax=ax[1])
plt.show()
fig, ax = plt.subplots(1, 2, figsize=(22, 3))
d = sns.distplot(clean_faces_pd["faces_min_conf"], kde=True, ax=ax[0])
d = sns.distplot(clean_faces_pd["faces_max_conf"], kde=True, ax=ax[1])
fig, ax = plt.subplots(figsize=(22, 3))
d = clean_faces_pd.plot(kind="scatter", x="faces_max_width", y="faces_max_conf", c="red", ax=ax, label="faces_max_width", alpha=0.5)
d = clean_faces_pd.plot(kind="scatter", x="faces_max_height", y="faces_max_conf", c="blue", ax=d,  label="faces_max_height", alpha=0.5)
d = plt.legend(loc="upper right")

In [ ]:
!pip install imutils

In [ ]:
import os
import numpy as np
import shutil
import cv2
import pandas as pd
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import numpy as np
import pickle
from imutils import paths
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model
from keras.optimizers import SGD
from sklearn.metrics import classification_report
%matplotlib inline

In [ ]:
BASE_PATH = "/kaggle/working/finetuningkeras/dataset"

# define the names of the training, testing, and validation
# directories
TRAIN = "training"
TEST = "evaluation"
VAL = "validation"

REAL = 'REAL'
FAKE = 'FAKE'

# initialize the list of class label names
CLASSES = ["FAKE", "REAL"]


# set the batch size when fine-tuning
BATCH_SIZE = 32

trainEpochs = 10
epochsFineTune = 10
maxVids = 5

# set the path to the serialized model after training
MODEL_PATH = os.path.sep.join(["/kaggle/working/finetuningkeras","output", "Deepfake.model"])
MODEL_PATH2 = os.path.sep.join(["/kaggle/working/finetuningkeras","output", "Deepfake2.h5"])
# define the path to the output training history plots
UNFROZEN_PLOT_PATH = os.path.sep.join(["/kaggle/working/finetuningkeras","output", "unfrozen.png"])
WARMUP_PLOT_PATH = os.path.sep.join(["/kaggle/working/finetuningkeras","output", "warmup.png"])

file = '/kaggle/input/deepfake-detection-challenge/train_sample_videos/metadata.json'
img_path = '/kaggle/input/deepfake-detection-challenge/train_sample_videos'
data_path = '/kaggle/working/finetuningkeras/real_fake'
dir_fake_frames = '/kaggle/working/FAKE_frames'
dir_real_frames = '/kaggle/working/REAL_frames'
dir_output = '/kaggle/working/finetuningkeras/output'

dir_data_path_real = os.path.join(data_path, REAL)
dir_data_path_fake = os.path.join(data_path, FAKE)

dir_train_real = os.path.join(BASE_PATH, TRAIN, REAL)
dir_train_fake = os.path.join(BASE_PATH, TRAIN, FAKE)
dir_valid_real = os.path.join(BASE_PATH, VAL, REAL)
dir_valid_fake = os.path.join(BASE_PATH, VAL, FAKE)
dir_test_real = os.path.join(BASE_PATH, TEST, REAL)
dir_test_fake = os.path.join(BASE_PATH, TEST, FAKE)

In [ ]:
input_dir = '/kaggle/working/finetuningkeras/real_fake/FAKE'
output_dir = '/kaggle/working/FAKE_frames/'
def explode_frames(input_dir, output_dir, maxN):

    mp4_filenames = [f for f in os.listdir(input_dir) if f.endswith('.mp4')]
    n = 0
    
    for mp4fn in mp4_filenames:
        
        if(n < maxN):
            n += 1 
            mp4fp = os.path.join(input_dir, mp4fn)
            cam = cv2.VideoCapture(mp4fp) 
            if(cam.isOpened()):
                print('Processing file #'+ str(n) + ' (' + mp4fn + ')...')
            else: 
                print('Problem opening file #'+ str(n) + ' (' + mp4fn + ')...')
                continue 
            
            nframe = 0
            while(True): #continue until ret = False then break
                nframe += 1
                ret,frame = cam.read()
                
                if ret: 
                    # if video is still left continue creating images 
                    out_filename = os.path.splitext(mp4fn)[0]+  '_frame' + str(nframe) + '.jpg'
                    out_filepath =  os.path.join(output_dir, out_filename)
                    
                    # writing the extracted images 
                    cv2.imwrite(out_filepath, frame) 
                else: 
                    break

            # Release all space and windows once done
            print(' - created ' + str(nframe-1) + ' images') # -1 bc count incremented before exit
            cam.release() 
            cv2.destroyAllWindows()
            
        else: 
            break
            
"""
Distribute files/images from a source directory into training, validation, and testing directories. 
src_dir = source/input directory
train_dir, val_dir, test_dir = target training/validation/testing directory
valperc = fraction of dataset to use for validation (0-1)
testperc = fraction of dataset to use for testing (0-1)
"""
            
def trainvaltest_split(src_dir, train_dir, val_dir, test_dir, valperc = 0.15, testperc = 0.15):
    
    filenames = os.listdir(src_dir) #get all filenames in random order
    np.random.shuffle(filenames)
    
    n = len(filenames)
    split1 = int(n*(1 - (valperc + testperc)))
    split2 = int(n*(1 - (testperc)))
    
    fn_train, fn_val, fn_test = np.split(np.array(filenames), [split1, split2])
    
    fn_lists = [fn_train, fn_val, fn_test]
    targetdirs = [train_dir, val_dir, test_dir]
    
    print('Total images: ', n)
    print('Training: ', len(fn_train))
    print('Validation: ', len(fn_val))
    print('Testing: ', len(fn_test))
    
    all_fp = [os.path.join(src_dir, fn) for fn in filenames]
    
    #move files
    for i, fn_list in enumerate(fn_lists):
        for fn in fn_list: 
            target_dir = targetdirs[i]
            fp_from = os.path.join(src_dir, fn)
            fp_to = os.path.join(target_dir, fn)
            
            shutil.move(fp_from, fp_to)

            
"""
Construct a plot that plots and saves the training history
"""           
def plot_training(H, N, plotPath):
	plt.style.use("ggplot")
	plt.figure()
	plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
	plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
	plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
	plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
	plt.title("Training Loss and Accuracy")
	plt.xlabel("Epoch #")
	plt.ylabel("Loss/Accuracy")
	plt.legend(loc="lower left")
	plt.savefig(plotPath)

In [ ]:
os.makedirs(dir_train_real, exist_ok = True)
os.makedirs(dir_train_fake, exist_ok = True)
os.makedirs(dir_valid_real, exist_ok = True)
os.makedirs(dir_valid_fake, exist_ok = True)
os.makedirs(dir_test_real, exist_ok = True)
os.makedirs(dir_test_fake, exist_ok = True)

os.makedirs(dir_data_path_real, exist_ok = True)
os.makedirs(dir_data_path_fake, exist_ok = True)
os.makedirs(dir_fake_frames, exist_ok = True) 
os.makedirs(dir_real_frames, exist_ok = True) 
os.makedirs(dir_output, exist_ok = True)

In [ ]:
df = pd.read_json(file)
df = df.T

# %% [code]
label = df[['label']]

In [ ]:
for fn, row in label.iterrows():
    src = os.path.join(img_path, fn)
    dest = os.path.join(data_path, row['label'], fn)
    shutil.copy(src, dest)a

In [ ]:
explode_frames(dir_data_path_fake, dir_fake_frames, maxN= maxVids)

In [ ]:
explode_frames(dir_data_path_real, dir_real_frames, maxN= maxVids)

In [ ]:
trainvaltest_split(src_dir = dir_fake_frames,
                   train_dir = dir_train_fake, 
                   val_dir = dir_valid_fake, 
                   test_dir = dir_test_fake)

In [ ]:
trainvaltest_split(src_dir = dir_real_frames,
                   train_dir = dir_train_real, 
                   val_dir = dir_valid_real, 
                   test_dir = dir_test_real)

In [ ]:

trainAug = ImageDataGenerator(
	rotation_range=30,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

In [ ]:
valAug = ImageDataGenerator()

In [ ]:
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
trainAug.mean = mean
valAug.mean = mean

In [ ]:
trainPath = os.path.join(BASE_PATH, TRAIN)
trainGen = trainAug.flow_from_directory(
	trainPath,
	class_mode="categorical",
	target_size=(224, 224),
	color_mode="rgb",
	shuffle=True,
	batch_size=BATCH_SIZE)

# initialize the validation generator
valPath = os.path.join(BASE_PATH, VAL)
valGen = valAug.flow_from_directory(
	valPath,
	class_mode="categorical",
	target_size=(224, 224),
	color_mode="rgb",
	shuffle=False,
	batch_size=BATCH_SIZE)

# initialize the testing generator
testPath = os.path.join(BASE_PATH, TEST)
testGen = valAug.flow_from_directory(
	testPath,
	class_mode="categorical",
	target_size=(224, 224),
	color_mode="rgb",
	shuffle=False,
	batch_size=BATCH_SIZE)

In [ ]:
baseModel = VGG16(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))

In [ ]:
headModel = baseModel.output
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(512, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(len(CLASSES), activation="softmax")(headModel)

In [ ]:
model = Model(inputs=baseModel.input, outputs=headModel)

In [ ]:
for layer in baseModel.layers:
	layer.trainable = False

In [ ]:
print("[INFO] compiling model...")
opt = SGD(lr=1e-4, momentum=0.9)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

In [ ]:
totalTrain = len(list(paths.list_images(trainPath)))
totalVal = len(list(paths.list_images(valPath)))
totalTest = len(list(paths.list_images(testPath)))

print("[INFO] training head...")
H = model.fit(
    trainGen,
    steps_per_epoch=totalTrain // BATCH_SIZE,
    validation_data=valGen,
    validation_steps=totalVal // BATCH_SIZE,
    epochs=trainEpochs)

In [ ]:
print("[INFO] evaluating after fine-tuning network head...")
testGen.reset()
predIdxs = model.predict_generator(testGen,
	steps=(totalTest // BATCH_SIZE) + 1)
predIdxs = np.argmax(predIdxs, axis=1)
print(classification_report(testGen.classes, predIdxs,
	target_names=testGen.class_indices.keys()))


plot_training(H, trainEpochs, WARMUP_PLOT_PATH)
plt.show()  

In [ ]:
from keras.utils import plot_model
plot_model(model, to_file='model.png')

In [ ]:
from sklearn.metrics import roc_curve, auc, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns 

In [ ]:
# Assuming testGen and model are defined
testGen.reset()
y_pred_probs = model.predict(testGen, steps=(totalTest // BATCH_SIZE) + 1)
y_true = testGen.classes

# Compute ROC curve for the positive class
fpr, tpr, _ = roc_curve(y_true, y_pred_probs[:, 1])
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=1, label='ROC curve (area = %0.2f)' % roc_auc)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

In [ ]:
testGen.reset()
y_pred_probs = model.predict(testGen, steps=(totalTest // BATCH_SIZE) + 1)

In [ ]:
# Compute predicted class labels
y_pred_labels = np.argmax(y_pred_probs, axis=1)

# Compute confusion matrix
cm = confusion_matrix(y_true, y_pred_labels)

# Plot confusion matrix
plt.figure(figsize=(5,5))
sns.heatmap(cm, annot=True, fmt="d")
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

In [ ]:
def plot_epoch_vs_accuracy(H, save_path=None):
    print("Starting to plot...")  # Debugging print statement
    epochs = len(H.history['accuracy'])  # Automatically determine the number of epochs
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, epochs + 1), H.history['accuracy'], label='Train Accuracy')
    plt.plot(range(1, epochs + 1), H.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Epoch vs Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend()
    
    if save_path:
        plt.savefig(save_path)
        
    plt.show()
    print("Plot should be displayed above.")  # Debugging print statement

# Assuming H is defined in your existing code
plot_epoch_vs_accuracy(H)


In [ ]:
trainGen.reset()
valGen.reset()

In [ ]:
for layer in baseModel.layers[15:]:
	layer.trainable = True

# loop over the layers in the model and show which ones are trainable
# or not
for layer in baseModel.layers:
	print("{}: {}".format(layer, layer.trainable))

# for the changes to the model to take affect we need to recompile
# the model, this time using SGD with a *very* small learning rate
print("[INFO] re-compiling model...")
opt = SGD(lr=1e-4, momentum=0.9)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

In [ ]:
H = model.fit_generator(
	trainGen,
	steps_per_epoch=totalTrain // BATCH_SIZE,
	validation_data=valGen,
	validation_steps=totalVal // BATCH_SIZE,
	epochs= epochsFineTune)

In [ ]:
print("[INFO] evaluating after fine-tuning network...")
testGen.reset()
predIdxs = model.predict_generator(testGen,
	steps=(totalTest // BATCH_SIZE) + 1)
predIdxs = np.argmax(predIdxs, axis=1)
print(classification_report(testGen.classes, predIdxs,
	target_names=testGen.class_indices.keys()))
plot_training(H, epochsFineTune, UNFROZEN_PLOT_PATH)

# serialize the model to disk
print("[INFO] serializing network...")
model.save(MODEL_PATH2)

In [ ]:
def plot_epoch_vs_accuracy(H, save_path=None):
    print("Starting to plot...")  # Debugging print statement
    epochs = len(H.history['accuracy'])  # Automatically determine the number of epochs
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, epochs + 1), H.history['accuracy'], label='Train Accuracy')
    plt.plot(range(1, epochs + 1), H.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Epoch vs Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend()
    
    if save_path:
        plt.savefig(save_path)
        
    plt.show()
    print("Plot should be displayed above.")  # Debugging print statement

# Assuming H is defined in your existing code
plot_epoch_vs_accuracy(H)


In [ ]:
# Compute predicted class labels
y_pred_labels = np.argmax(y_pred_probs, axis=1)

# Compute confusion matrix
cm = confusion_matrix(y_true, y_pred_labels)

# Plot confusion matrix
plt.figure(figsize=(5,5))
sns.heatmap(cm, annot=True, fmt="d")
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()


In [ ]:
from keras.utils import plot_model
plot_model(model, to_file='model.png')